# Notebook to solve the kaggle competition "Child Mind Institute - Detect Sleep States"
Link to the competition: https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states


### Install packages

In [1]:
%pip install matplotlib plotly pandas numpy tqdm scikit-learn pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [2]:
from itertools import product
from tqdm import tqdm
from score import score
#from sklearn.svm import SVC
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, average_precision_score, f1_score
import wandb
from abc import ABC, abstractmethod #, classmethod
from math import ceil, floor
import os
from pathlib import Path
from joblib import dump, load
from custom_enums import ModelTrainingType
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import gc
from scipy.special import softmax

In [3]:

# wandb.login() requires you to get your API key from your account settings
# open the weights and biases website https://wandb.ai/login and login to your account
# then go to your account settings and copy the API key
# paste it in the input box and hit enter

#wandb.login() #TODO: uncomment this line to login to your account

In [4]:
# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

To be able to switch between different models from different libraries at a glance, we implement an interface called `IPipelineRequirements`. This allows us to make the pipleine even more robust and easier to extend upon. 

In [5]:
class IPipelineRequirements(ABC):
    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def predict(self):
        pass

    @abstractmethod
    def save(self):
        pass

    @abstractmethod
    def load(self):
        pass

    @abstractmethod
    def evaluate(self):
        pass

Our first model is the baseline model, which just takes the mean over all onset and wakeup times and tries to predict `onset` and `wakeup` events with the calculated time.

In [6]:
class BaselineModel(IPipelineRequirements):
    def __init__(self):
        pass

    def train(self):
        raise NotImplementedError("Please Implement this method")

    def predict(self):
        raise NotImplementedError("Please Implement this method")

    def save(self):
        raise NotImplementedError("Please Implement this method")

    def load(self):
        raise NotImplementedError("Please Implement this method")

    def evaluate(self):
        raise NotImplementedError("Please Implement this method")

In [7]:
g = BaselineModel()

Numerous studies have concentrated on applications using `RandomForest`. The primary motivation for this preference is the model's inherent transparency in decision-making processes, which are readily identifiable in such models. Subsequent to the BaselineModel, efforts have been made to abstract models from the Scikit-learn library. Fortunately, the majority of models within their API exhibit consistent implementation patterns, facilitating their integration into the processing pipeline.

In [8]:
class SkLearnModel(IPipelineRequirements):

    def __init__(self, model_path):
        self.load(model_path)

    def __init__(self, model, identifier, scaler=StandardScaler, sk_model_params=None):
        self._model = model
        self._scaler = scaler if not callable(scaler) else scaler() # if scaler is a classpointer, instantiate it
        self.identifier = identifier
        self._model_params = sk_model_params

    def train(self, X, y, not_scaled=False, **kwargs):
        if not_scaled:
            X = self._scaler.transform(X)

        # due to the big dataset we need to check wich model was instantiated and do some model
        # specific stuff to enable us to train the model in batches
        if isinstance(self._model, RandomForestClassifier):
            new_estimators = kwargs['add_estimators'] if 'add_estimators' in kwargs else 50
            if self._model.warm_start: self._model.n_estimators += new_estimators
            #print(f'Current estimator increased to {self._model.n_estimators}, {new_estimators} added this round.')
        # if isinstance(self._model, SVC):
        #     pass

        self._model.fit(X, y)

    def predict(self, X, not_scaled=False):
        if not self._model: raise ValueError('Please load or train a model first.')
        if not_scaled:
            if not self._scaler: raise ValueError('Please load or fit a scaler first.')
            X = self._scaler.transform(X)
        return self._model.predict(X)

    def save(self):
        try:
            Path.mkdir(Path('models'), exist_ok=True)
            with open(f'models/model_{self.identifier}.jlb', 'wb') as f:
                dump(self._model, f)
            with open(f'models/scaler_{self.identifier}.jlb', 'wb') as f:
                dump(self._scaler, f)
        except:
            raise ValueError('Unable to save model and scaler.')

    def load(self, filepath):
        try:
            # load model and scaler
            if os.path.isfile(filepath):
                print(f'Loading model from {filepath}')
                with open(filepath, 'rb') as f:
                    self._model = load(f)
            scaler_path = f'{os.path.split(filepath)[0]}/scaler_{os.path.split(filepath)[1].split(".")[0].split("_")[1]}.pkl'
            if os.path.isfile(scaler_path):
                print(f'Loading scaler from {scaler_path}')
                with open(filepath, 'rb') as f:
                    self._scaler = load(f)

            # extract identifier from filename
            self._identifier = filepath.split('_')[1].split('.')[0]
        except (FileNotFoundError) as e:
                print(f'File {e} not found')
                raise ValueError('Filepath is not valid. Unable to load model and scaler.')
        except (IndexError) as e:
                print(f'The name of the file does not implement the convention "<model|scaler>_<some identifier>".')
                raise ValueError('Filepath is not valid. Unable to load model and scaler.')
        except (Exception) as e:
            print(f'Something went wrong. {e}')
            raise ValueError('Unknown Error.')

    def evaluate(self, X, y, scoreFx=None, not_scaled=False, prepredicted=None):
        if not scoreFx:
            raise ValueError('Please provide a score function.')
        if not self._model:
            raise ValueError('Please load or train a model first.')
        if not_scaled and prepredicted is None:
            if not self._scaler: raise ValueError('Please load or fit a scaler first.')
            X = self._scaler.transform(X)

        return scoreFx(y, prepredicted if prepredicted is not None else self._model.predict(X))

### Heuristics method to get the right event under the constraints given by Child Mind Institue

* A single sleep period must be at least 30 minutes in length
* A single sleep period can be interrupted by bouts of activity that do not exceed 30 consecutive minutes
* No sleep windows can be detected unless the watch is deemed to be worn for the duration (elaborated upon, below)
* The longest sleep window during the night is the only one which is recorded
* If no valid sleep window is identifiable, neither an onset nor a wakeup event is recorded for that night.
* Sleep events do not need to straddle the day-line, and therefore there is no hard rule defining how many may occur within a given period. However, no more than one window should be assigned per night. For example, it is valid for an individual to have a sleep window from 01h00–06h00 and 19h00–23h30 in the same calendar day, though assigned to consecutive nights
* There are roughly as many nights recorded for a series as there are 24-hour periods in that series.

In [9]:
def heuristic_filter(data):
    pass

### Import the training and the validation data

In [10]:
raise(ValueError('test'))

ValueError: test

In [11]:
NEW_TRAIN_DATA = '../data/train/'
NEW_VALIDATION_DATA = '../data/val/'
TRAIN_DATA = '../data/train_20231021'
#TRAIN_DATA = '../data/train_20231021_20M'
VALIDATION_DATA = '../data/validation_20231021'

# figure out series id mapping
if not 'series_id_mapping' in vars():
    series_id_mapping = {'train': dict(), 'validation': dict()}
    t = ds.dataset(NEW_TRAIN_DATA).to_table(columns=['series_id'])
    #t = pq.ParquetDataset(TRAIN_DATA).read(columns=['series_id'])
    for i, data in enumerate(t.to_pandas()['series_id'].unique()):
        series_id_mapping['train'][data] = i
    #v = pq.ParquetDataset(VALIDATION_DATA).read(columns=['series_id'])
    v = ds.dataset(NEW_VALIDATION_DATA).to_table(columns=['series_id'])
    for i, data in enumerate(v.to_pandas()['series_id'].unique()):
        series_id_mapping['validation'][data] = i
    del t, v
    gc.collect()

#train_dataset_length = pq.ParquetFile(TRAIN_DATA).metadata.num_rows
train_dataset_length = ds.dataset(NEW_TRAIN_DATA).count_rows()
validation_dataset_length = ds.dataset(NEW_VALIDATION_DATA).count_rows()

def dataloader_full_dataset(validation=False):
    return pd.read_parquet(VALIDATION_DATA if validation else TRAIN_DATA)

def dataloader(validation=False, batch_size=5_000_000):
    parquet_file = pq.ParquetFile(VALIDATION_DATA if validation else TRAIN_DATA)
    for i in parquet_file.iter_batches(batch_size=batch_size):
        yield i.to_pandas()

def batched_dataloader(validation=False, batch_size=100_000):
    dataset = ds.dataset(NEW_VALIDATION_DATA if validation else NEW_TRAIN_DATA)
    batch = pd.DataFrame()
    for file_batch in dataset.to_batches(batch_size=batch_size):
        batch = pd.concat([batch, file_batch.to_pandas()])
        if len(batch) >= batch_size:
            yield batch.reset_index(drop=True)
            batch = pd.DataFrame()
    yield batch.reset_index(drop=True)

    # for fragment in dataset.get_fragments():
    #     batches = fragment.to_batches(batch_size=batch_size) # https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Fragment.html#pyarrow.dataset.Fragment.to_batches
    #     for batch in batches:
    #         yield batch.to_pandas()
    #         del batch
    #     del fragment


### Run the pipeline

In [12]:
# define all the combinations of models and features
models_and_hyperparams = {
    RandomForestClassifier: {
        'params': {
            'n_estimators': [500, 600, 400],
            'max_depth': [None, 20, 40, 60],
            'min_samples_leaf': [5, 15, 25],
            'random_state': [0],
            'n_jobs': [10],
            #'criterion': ['gini', 'entropy', 'log_loss'],
            'warm_start': [True]
        },
        'modeltype': SkLearnModel,
        'scaler': StandardScaler,
        'training_type': ModelTrainingType.BATCH,
        'batch_size': 3_000_000
    }
}

# define scaler
scaler_type = StandardScaler
pretrained_scalers = dict() # used to store trained scalers for later use

# define columns to drop while training the model
additionally_drop = ['anglez_rolling_mean_1', 'anglez_rolling_max_1', 'anglez_rolling_std_1',
       'anglez_diff_1', 'anglez_diff_rolling_mean_1',
       'anglez_diff_rolling_max_1', 'anglez_diff_rolling_std_1', 'enmo_rolling_max_1', 'enmo_rolling_std_1', 'enmo_diff_1',
       'enmo_diff_rolling_mean_1', 'enmo_diff_rolling_max_1',
       'enmo_diff_rolling_std_1']
columns_to_drop = ['series_id', 'step', 'awake', 'wearable_on', 'seconds', 'year', 'anglez_abs', 'enmo_abs'] + additionally_drop

# loop over all combinations and append it to the configurations list
# if there are no hyperparams, just instantiate the model without params
for model_type, hyperparams in models_and_hyperparams.items():
    if len(hyperparams['params']) > 0: # if there are hyperparams, build a dict and pass it to the model as parameters
        fx_param_names, fx_param_values = zip(*hyperparams['params'].items())
        for cartesian_product_values in product(*fx_param_values):
            hyperparam_dict = dict(zip(fx_param_names, cartesian_product_values))

            # create or reuse the scaler specified in the models_and_hyperparams dictionary
            if hyperparams['scaler'] in pretrained_scalers:
                print(f'Using pretrained scaler {hyperparams["scaler"].__name__}')
                scaler = pretrained_scalers[hyperparams['scaler']]
            else:
                scaler = hyperparams['scaler']()
                print(f'Start fitting scaler {hyperparams["scaler"].__name__}')
                for batch in tqdm(batched_dataloader(batch_size=hyperparams['batch_size']) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset()):
                    X = batch.drop(columns_to_drop, axis=1)
                    #X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
                    #X['series_id'] = X['series_id'].map(series_id_mapping['train'])
                    scaler.partial_fit(X)
                    del X
                    gc.collect()
                # save the scaler for later use
                pretrained_scalers[hyperparams['scaler']] = scaler

            # create the model from the modeltype and the hyperparam_dict
            m = hyperparams['modeltype'](model_type(**hyperparam_dict), \
                                        f'{model_type.__name__}-wrist-{"-".join([f"{na}__{str(va)}" for na, va in hyperparam_dict.items()])}'.lower(), \
                                        scaler, \
                                        #StandardScaler(), \
                                        hyperparam_dict)
            m2 = hyperparams['modeltype'](model_type(**hyperparam_dict), \
                                        f'{model_type.__name__}-sleep-{"-".join([f"{na}__{str(va)}" for na, va in hyperparam_dict.items()])}'.lower(), \
                                        scaler, \
                                        #StandardScaler(), \
                                        hyperparam_dict)

            # init wandb
            # start a new wandb run to track this script
            #cfg = m._model_params.copy()
            #cfg['model'] = m._model.__name__
            #wandb.init(project="classic_models", config=cfg)

            # train model
            rounds = 1
            if hyperparams['training_type'] == ModelTrainingType.BATCH:
                print(f'Model will be traind in batches of {hyperparams["batch_size"]} samples.')
                print(f'Dataset contains {train_dataset_length} samples. Batchsize is {hyperparams["batch_size"]}. That means {(rounds := train_dataset_length / hyperparams["batch_size"])} batches will be used to fit the model.')
                if model_type == RandomForestClassifier:
                    # print(f'At the end {hyperparam_dict["n_estimators"]} estimators will be fitted. That means, {hyperparam_dict["n_estimators"]//rounds} estimators per batch.')
                    m._model.n_estimators = 0 #set to 0. model will increase the number of estimators in each round
                    m2._model.n_estimators = 0 #set to 0. model will increase the number of estimators in each round
                    # # tune hyperparameter 'n_estimators' based on the number of batches
                    # attune_estimators_div = hyperparam_dict["n_estimators"] / rounds
                    # attuned_estimators = (floor(attune_estimators_div) if attune_estimators_div -0.3 < floor(attune_estimators_div) else ceil(attune_estimators_div)) * rounds
                    # print(f'Hyperparameter n_estimators will be adjusted to {attuned_estimators} to fit {attuned_estimators / rounds} estimators per round for each {rounds} batches.')
                    # hyperparam_dict["n_estimators"] = attuned_estimators
                    print(f'Build estimator distribution for {rounds} rounds.')
                    est_anteil = [1 for _ in range(floor(rounds))]
                    est_anteil.append(rounds % floor(rounds))
                    est_anteil = softmax(est_anteil)
                    est_anteil *= hyperparam_dict['n_estimators']
                    est_anteil = est_anteil.round().astype(int)
                    print(f'Estimator distribution per round: {est_anteil}')

            print(f'Start training model {m.identifier}')
            for i, batch in enumerate(tqdm(batched_dataloader(batch_size=hyperparams['batch_size']) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(), total=floor(rounds)+1)):
                y = batch['wearable_on']
                X = batch.drop(columns_to_drop, axis=1)
                #X = batch.drop(['wearable_on', 'awake'], axis=1)
                #X['series_id'] = X['series_id'].map(series_id_mapping['train'])
                args = {}
                if model_type == RandomForestClassifier and hyperparams['training_type'] == ModelTrainingType.BATCH: args['add_estimators'] = est_anteil[i]
                m.train(X, y, not_scaled=True, **args)
                del X
                del y
                gc.collect()

            #  evaluate model
            print(f'Start evaluating model {m.identifier}')
            score_value_average_precision_score, score_value_recall, score_value_precision, score_f1 = [], [], [], []
            for validation in tqdm(batched_dataloader(batch_size=hyperparams['batch_size'], validation=True) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(validation=True), total = ceil(validation_dataset_length / hyperparams["batch_size"])):
                #validation['series_id'] = validation['series_id'].map(series_id_mapping['validation'])
                validation_y = validation['wearable_on']
                validation.drop(columns_to_drop, axis=1, inplace=True)
                #validation.drop(['wearable_on', 'awake'], axis=1, inplace=True)
                validation_y_hat = m.predict(validation, not_scaled=True)
                # this does the same as scoreFX(y, y_hat). It is wrapped in the models evaluate function.
                # the model itself could also do the prediction if prepredicted is None
                # then, the first argument of the evaluate function would be the X_validation data
                score_value_average_precision_score.append(m.evaluate(None, validation_y, scoreFx=average_precision_score, not_scaled=True, prepredicted=validation_y_hat))
                score_value_recall.append(m.evaluate(None, validation_y, scoreFx=recall_score, not_scaled=True, prepredicted=validation_y_hat))
                score_value_precision.append(m.evaluate(None, validation_y, scoreFx=precision_score, not_scaled=True, prepredicted=validation_y_hat))
                score_f1.append(m.evaluate(None, validation_y, scoreFx=f1_score, not_scaled=True, prepredicted=validation_y_hat))
                del validation, validation_y

            print(sum(score_value_average_precision_score) / len(score_value_average_precision_score))
            print(sum(score_value_recall) / len(score_value_recall))
            print(sum(score_value_precision) / len(score_value_precision))
            print(sum(score_f1) / len(score_f1))
            #  save model
            m.save()

            print(f'Start training model {m2.identifier}')
            for i, batch in enumerate(tqdm(batched_dataloader(batch_size=hyperparams['batch_size']) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(), total=floor(rounds)+1)):
                y = batch['awake']
                X = batch.drop(columns_to_drop, axis=1)
                #X = batch.drop(['wearable_on', 'awake'], axis=1)
                #X['series_id'] = X['series_id'].map(series_id_mapping['train'])
                args = {}
                if model_type == RandomForestClassifier and hyperparams['training_type'] == ModelTrainingType.BATCH: args['add_estimators'] = est_anteil[i]
                worn_y_hat = m.predict(X, not_scaled=True)
                # add worn_y_hat as new column to X
                X['pred_worn'] = worn_y_hat
                # filter out all rows where worn_y_hat is 0
                X = X[X['pred_worn'] == 1]
                y = y[y.index.isin(X.index)]
                X = X.drop(['pred_worn'], axis=1)
                m2.train(X, y, not_scaled=True, **args)
                del X, y
                gc.collect()

            print(f'Start evaluating model {m2.identifier}')
            score_value_average_precision_score_m2, score_value_recall_m2, score_value_precision_m2, f1_score_m2 = [], [], [], []
            for validation in tqdm(batched_dataloader(batch_size=hyperparams['batch_size'], validation=True) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(validation=True), total = ceil(validation_dataset_length / hyperparams["batch_size"])):
                #validation['series_id'] = validation['series_id'].map(series_id_mapping['validation'])
                validation_y = validation['awake']
                validation.drop(columns_to_drop, axis=1, inplace=True)
                #validation.drop(['wearable_on', 'awake'], axis=1, inplace=True)
                worn_y_hat = m.predict(validation, not_scaled=True)
                validation['pred_worn'] = worn_y_hat
                validation = validation[validation['pred_worn'] == 1]
                validation_y = validation_y[validation_y.index.isin(validation.index)]
                validation = validation.drop(['pred_worn'], axis=1)
                validation_y_hat = m2.predict(validation, not_scaled=True)
                # this does the same as scoreFX(y, y_hat). It is wrapped in the models evaluate function.
                # the model itself could also do the prediction if prepredicted is None
                # then, the first argument of the evaluate function would be the X_validation data
                score_value_average_precision_score_m2.append(m2.evaluate(None, validation_y, scoreFx=average_precision_score, not_scaled=True, prepredicted=validation_y_hat))
                score_value_recall_m2.append(m2.evaluate(None, validation_y, scoreFx=recall_score, not_scaled=True, prepredicted=validation_y_hat))
                score_value_precision_m2.append(m2.evaluate(None, validation_y, scoreFx=precision_score, not_scaled=True, prepredicted=validation_y_hat))
                f1_score_m2.append(m2.evaluate(None, validation_y, scoreFx=f1_score, not_scaled=True, prepredicted=validation_y_hat))

                del validation, validation_y

            print(sum(score_value_average_precision_score_m2) / len(score_value_average_precision_score_m2))
            print(sum(score_value_recall_m2) / len(score_value_recall_m2))
            print(sum(score_value_precision_m2) / len(score_value_precision_m2))
            print(sum(f1_score_m2) / len(f1_score_m2))
            #  save model
            m2.save()

            break

            #wandb.finish()
    else:
        print("params in the dictionary cannot be empty. Use the standard values in the dictionary for the model", model_type.__name__)

Start fitting scaler StandardScaler


0it [00:00, ?it/s]

3it [00:34, 11.66s/it]


Model will be traind in batches of 3000000 samples.
Dataset contains 8695770 samples. Batchsize is 3000000. That means 2.89859 batches will be used to fit the model.
Build estimator distribution for 2.89859 rounds.
Estimator distribution per round: [172 172 156]
Start training model randomforestclassifier-wrist-n_estimators__500-max_depth__none-min_samples_leaf__5-random_state__0-n_jobs__8-warm_start__true


100%|██████████| 3/3 [52:18<00:00, 1046.25s/it]


Start evaluating model randomforestclassifier-wrist-n_estimators__500-max_depth__none-min_samples_leaf__5-random_state__0-n_jobs__8-warm_start__true


100%|██████████| 1/1 [00:22<00:00, 22.12s/it]


0.7797600992844761
0.9905153702082475
0.7799564741645187
0.8727152348780627
Start training model randomforestclassifier-sleep-n_estimators__500-max_depth__none-min_samples_leaf__5-random_state__0-n_jobs__8-warm_start__true


  0%|          | 0/3 [00:39<?, ?it/s]


ValueError: Found input variables with inconsistent numbers of samples: [2668696, 2989508]

In [23]:
X

,anglez,enmo,hour,minute,day,month,anglez_rolling_mean_5,anglez_rolling_max_5,anglez_rolling_std_5,anglez_diff_5,...,enmo_diff_rolling_mean_120,enmo_diff_rolling_max_120,enmo_diff_rolling_std_120,enmo_rolling_mean_480,enmo_rolling_max_480,enmo_rolling_std_480,enmo_diff_480,enmo_diff_rolling_mean_480,enmo_diff_rolling_max_480,enmo_diff_rolling_std_480
0,2.689800,0.021775,15,30,14,8,-35.181107,28.072075,48.950768,-82.706924,...,0.186867,0.186867,0.000000,0.050112,0.485575,0.061099,-0.003450,-0.003450,-0.003450,0.000000
1,28.072075,0.020275,15,31,14,8,-35.181107,28.072075,48.950768,-82.706924,...,0.186867,0.186867,0.000000,0.050112,0.485575,0.061099,-0.003450,-0.003450,-0.003450,0.000000
2,-46.665344,0.017383,15,32,14,8,-35.181107,28.072075,48.950768,-82.706924,...,0.186867,0.186867,0.000000,0.050112,0.485575,0.061099,-0.003450,-0.003450,-0.003450,0.000000
3,-79.990448,0.013625,15,33,14,8,-51.722492,28.072075,46.884663,-82.706924,...,0.186867,0.186867,0.000000,0.050112,0.485575,0.061099,-0.003450,-0.003450,-0.003450,0.000000
4,-80.011620,0.013650,15,34,14,8,-73.345123,-46.665344,14.914462,-82.706924,...,0.186867,0.186867,0.000000,0.050112,0.485575,0.061099,-0.003450,-0.003450,-0.003450,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668691,18.489117,0.067817,7,25,17,1,18.394241,41.092007,15.486051,-0.873816,...,0.001474,0.095100,0.024666,0.015374,0.153333,0.015548,0.067817,-0.049056,0.116717,0.087793
2668692,21.705942,0.064800,7,26,17,1,25.532625,41.092007,11.848199,-32.062057,...,0.001474,0.095100,0.024666,0.015374,0.153333,0.015548,0.064800,-0.049056,0.116717,0.087793
2668693,41.092007,0.068008,7,27,17,1,29.990015,41.092007,9.511382,5.569221,...,0.001474,0.095100,0.024666,0.015374,0.153333,0.015548,0.068008,-0.049056,0.116717,0.087793
2668694,34.237377,0.050450,7,28,17,1,29.990015,41.092007,9.511382,35.691917,...,0.001474,0.095100,0.024666,0.015374,0.153333,0.015548,0.050450,-0.049056,0.116717,0.087793


In [18]:
pd.DataFrame(validation_y_hat).value_counts()

1    1894507
0      70298
Name: count, dtype: int64

In [ ]:
print(sum(score_value_average_precision_score_m2) / len(score_value_average_precision_score_m2))
print(sum(score_value_recall_m2) / len(score_value_recall_m2))
print(sum(score_value_precision_m2) / len(score_value_precision_m2))

In [ ]:
with open('models/AAAA.pkl', 'wb') as f:
    dump(m._model, f)

In [ ]:
with open('models/BBBB.pkl', 'wb') as f:
    dump(m2._model, f)

In [ ]:
m._model.feature_importances_

In [13]:
ff = batched_dataloader(validation=True, batch_size=1)

In [14]:
fg = next(ff)

In [15]:
fg.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
fg['pred_worn'] = 0

In [16]:
fg.columns


Index(['anglez', 'enmo', 'hour', 'minute', 'day', 'month',
       'anglez_rolling_mean_5', 'anglez_rolling_max_5', 'anglez_rolling_std_5',
       'anglez_diff_5', 'anglez_diff_rolling_mean_5',
       'anglez_diff_rolling_max_5', 'anglez_diff_rolling_std_5',
       'anglez_rolling_mean_30', 'anglez_rolling_max_30',
       'anglez_rolling_std_30', 'anglez_diff_30',
       'anglez_diff_rolling_mean_30', 'anglez_diff_rolling_max_30',
       'anglez_diff_rolling_std_30', 'anglez_rolling_mean_120',
       'anglez_rolling_max_120', 'anglez_rolling_std_120', 'anglez_diff_120',
       'anglez_diff_rolling_mean_120', 'anglez_diff_rolling_max_120',
       'anglez_diff_rolling_std_120', 'anglez_rolling_mean_480',
       'anglez_rolling_max_480', 'anglez_rolling_std_480', 'anglez_diff_480',
       'anglez_diff_rolling_mean_480', 'anglez_diff_rolling_max_480',
       'anglez_diff_rolling_std_480', 'enmo_rolling_mean_1',
       'enmo_rolling_mean_5', 'enmo_rolling_max_5', 'enmo_rolling_std_5',
      

In [17]:
sorted(list(zip(fg.columns, m._model.feature_importances_.round(4))), key=lambda x: x[1], reverse=True)

[('anglez_diff_rolling_max_480', 0.0862),
 ('anglez_diff_rolling_std_480', 0.0643),
 ('anglez_rolling_max_480', 0.0625),
 ('anglez_rolling_std_480', 0.0546),
 ('month', 0.051),
 ('enmo_diff_rolling_std_480', 0.046),
 ('day', 0.045),
 ('enmo_rolling_max_480', 0.0437),
 ('anglez_diff_rolling_std_120', 0.0388),
 ('anglez_rolling_std_120', 0.0373),
 ('enmo_diff_rolling_max_480', 0.0362),
 ('anglez_diff_rolling_max_120', 0.0332),
 ('anglez_rolling_max_120', 0.0302),
 ('anglez_rolling_mean_480', 0.0293),
 ('enmo_rolling_std_480', 0.0269),
 ('anglez_diff_rolling_std_30', 0.0222),
 ('anglez_diff_rolling_mean_480', 0.0213),
 ('enmo_diff_rolling_mean_480', 0.0198),
 ('enmo_rolling_mean_480', 0.0187),
 ('enmo_rolling_max_120', 0.0179),
 ('enmo_diff_rolling_std_120', 0.0168),
 ('anglez_rolling_mean_120', 0.0163),
 ('enmo_diff_rolling_max_120', 0.0163),
 ('enmo_rolling_std_120', 0.0136),
 ('enmo_rolling_mean_120', 0.0132),
 ('anglez_rolling_std_30', 0.0124),
 ('hour', 0.0121),
 ('anglez_rolling_max

In [ ]:
raise ValueError('test')

1. add output of the **BEST** model 1 to train dataset -> [pred_wearable_on] -> must be saved to disk in batches!!!
2. feed the new training to the second model
   1. x of the second model is == ALL but X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
   2. y of the second model is y = batch['awake']

- Do not forget to map the series ID
- All values where pred_wearable_on == 0 must be dropped from the traning set for the second model...
  this can be done by adding a filter to the parquet reader like so:
  ```python
  table = pq.read_table("example.parquet",
                      columns=["col1"],
                      filters=[
                          ("col1", ">", 5),
                          ("col1", "<", 10),
                      ])
  ```


1. the output of the **BEST** second model [pred_awake] must be passed through the heuristics fx alongside with.
   1. the following datafields ['series_id', 'ALL TIME FIELDS', 'pred_awake, ??? others???] 


In [ ]:
#TODO: https://arrow.apache.org/cookbook/py/io.html#id11
# for splitted files 